# INELASTIC TOY NETWORK STA

This file first shows how to run an inelastic Static Traffic Assignment in dyntapy. It also shows examples of how to run an inelastic assignment with tolls.

1. At first, you import all the necessary packages and files from mostly dyntapy. 

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import numpy as np
import os
import sys
sys.path.append("../../..")

from pickle import dump
from dyntapy.supply_data import get_toy_network, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network, show_demand
from dyntapy.toll import create_toll_object

2. Some toy networks are already made available in the original toolbox, so in the example, Cascetta is used. The first steps are always the same: 

- Create or retrieve a network
- Create and add centroids to network → adds connectors automatically
- Create or load an OD matrix (with the demand) onto the centroids

  These steps can also be visualised in the notebooks by fully running the next block of code. There are now links visible (roads and connectors), green centroids and for both elements when hovering with the cursor over the segments there are also some characteristics displayed in a box like the link ID, the capacity, the free-flow speed or demand between each location in the demand plot.

In [ ]:
# 1. Retrieve network 
g = get_toy_network('cascetta')

# 2. Create and add centroids
centroid_x = np.array([2+np.sqrt(2), 5, np.sqrt(2)])
centroid_y = np.array([-0.5, np.sqrt(2), np.sqrt(2)])
g = add_centroids(g, centroid_x, centroid_y, euclidean=True)
# also adds connectors automatically
g = relabel_graph(g)  # adding link and node ids, connectors and centroids
show_network(g, euclidean=True, notebook=True)

# 3. Create OD and load onto centroids 
od_matrix = np.zeros(9).reshape((3, 3))
od_matrix[0, 1] = 100
od_matrix[2, 1] = 100
od_graph = od_graph_from_matrix(od_matrix, centroid_x, centroid_y)
show_demand(od_graph, euclidean=True, notebook=True)

3. Now all the elements are there to run an STA in the next block of code. When this assignment is performed, there is now a loaded network visible with the traffic flows as an additional characteristic. 

In [ ]:
assignment = StaticAssignment(g, od_graph)
result = assignment.run('dial_b')
print('dial_b ran successfully')
show_network(g, flows = result.flows, notebook=True, show_nodes=False, euclidean=True)

    Congratulations, you have now run your first STA using dyntapy!

4. To now add toll, please first take a look at the “tolling” folder inside the case studies. Here you will see examples of all tolling methods, including single link, cordon- and zone-based tolling schemes. To get to know more about what these mean, consult the literature or the additional paper. What is shown in the code are the three main elements per scheme: the links where the toll is put on, the value of this toll and the toll_object, which is the additional element that should be given to the STA. Decide on the links and the toll values and afterwards, please run the StaticAssignment again in the next block of code. You now have the distribution of flows with the toll cost on top of the normal BPR link costs.

In [ ]:
# Create toll object
toll_method = 'zone'
toll_link_ids = [9,10] # Link 9 goes from the bottom to the right, and link 10 goes from the bottom to the top. 
toll_value = 3
toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)

# Run STA with toll object
assignment2 = StaticAssignment(g,od_graph, toll_object)
result2 = assignment2.run('dial_b')
print('dial_b ran successfully')
show_network(g, flows = result2.flows, notebook=True, show_nodes=False, euclidean=True)


    Congratulations, you have now implemented toll on a toy network! 

-----------------------------------------------------------------------------------------------------------------------


5. The last step is preparing for the Heeds iterations. The outer loop will decide on the best toll value, by ‘randomly’ trying toll values within a given range. To do this, the full network (containing roads, centroids and connectors) with the OD matrix should preferably be saved to avoid reloading these files each iteration. Because both remain static in an inelastic use-case, they are saved with a given path. The method to do so is indicated in the last block of the file.

In [ ]:
HERE = os.path.dirname(os.path.realpath("__file__"))
HEEDS_data_path = HERE+ os.path.sep+os.pardir+os.path.sep+os.pardir+os.path.sep+'data_map' + os.path.sep + "HEEDS_input"

network_path = HEEDS_data_path + os.path.sep + 'network_with_centroids' + os.path.sep + "inelastic_toy"
graph_path = HEEDS_data_path + os.path.sep + "od_graph" + os.path.sep + "inelastic_toy"

with open(network_path, 'wb') as network_file:
    dump(g, network_file)
    print(f'network saved at f{network_path}')
with open(graph_path, 'wb') as f:
    dump(od_graph,f)
    print(f'od_graph saved at f{graph_path}')

    Congratulations, you are now ready to go to the next step: HEEDS!